# Tool-based Investment Agent (ReAct Pattern)

## 아키텍처 개선:
- ✅ **Idris 명세 기반 구현** (`idris/Domain/Tools.idr`, `idris/Domain/ReActAgent.idr`)
- ✅ **LangChain Tool 프레임워크** 사용
- ✅ **ReAct 패턴** (Reasoning + Acting)
- ✅ **확장 가능한 도구 시스템**
- ✅ **표준 LangGraph prebuilt components**

## Idris 명세와의 대응:
- `python/models/tools.py` ← `idris/Domain/Tools.idr`
- 이 노트북 ← `idris/Domain/ReActAgent.idr`

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

## 1. Tool 가져오기 (Tools.idr 기반 구현)

In [ ]:
import sys
sys.path.append('..')

from python.models.tools import (
    AVAILABLE_TOOLS,
    search_web,
    get_stock_price,
    calculate_moving_average,
    get_company_info,
    ToolAgentState
)

print(f"✅ {len(AVAILABLE_TOOLS)}개의 도구 로드 완료:")
for t in AVAILABLE_TOOLS:
    print(f"  - {t.name}: {t.description}")

## 2. ReAct Agent 생성 (ReActAgent.idr 기반)

Idris spec: `PrebuiltReActAgent` in `ReActAgent.idr`

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 시스템 프롬프트 (Idris spec: systemPrompt in PrebuiltReActAgent)
system_prompt = """당신은 전문 주식 투자 분석가입니다.

**역할**:
- 사용자의 투자 관련 질문에 데이터 기반으로 답변
- 필요한 경우 제공된 도구를 사용하여 정보 수집
- 실시간 주가, 기업 정보, 뉴스 등을 활용한 분석

**사용 가능한 도구**:
1. search_web: 웹에서 최신 뉴스 및 정보 검색
2. get_stock_price: 특정 주식의 가격 정보 조회
3. calculate_moving_average: 기술적 분석 (이동평균선)
4. get_company_info: 기업 기본 정보 및 재무 지표

**답변 원칙**:
- 구체적인 데이터와 출처를 제시
- 불확실한 정보는 명시
- 투자 결정은 사용자의 몫임을 강조
"""

# ReAct Agent 생성 (Idris spec: createAgent in ReActAgent.idr)
agent = create_react_agent(
    llm,
    AVAILABLE_TOOLS,
    state_modifier=system_prompt
)

print("✅ ReAct Agent 생성 완료")
print(f"   도구 개수: {len(AVAILABLE_TOOLS)}")
print(f"   LLM 모델: {llm.model_name}")

## 3. 그래프 시각화

Idris spec: `AgentNode` (LLMNode, ToolsNode) in `ReActAgent.idr`

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"그래프 시각화 실패: {e}")
    print("\n텍스트 구조:")
    print("START → __start__ → agent → tools → agent → __end__ → END")

## 4. 헬퍼 함수

Idris spec: `ExecutionCycle` and streaming in `ReActAgent.idr`

In [ ]:
def run_agent(query: str, verbose: bool = True):
    """
    Agent 실행 헬퍼 함수
    
    Idris spec:
    - AgentExecutionState in ReActAgent.idr
    - ExecutionCycle (AgentThinking, ToolsExecuting, Completed)
    """
    print(f"\n{'='*70}")
    print(f"질문: {query}")
    print(f"{'='*70}\n")
    
    messages = [{"role": "user", "content": query}]
    
    # Idris spec: stream with StreamMode (Values, Updates, Messages)
    for chunk in agent.stream({"messages": messages}, stream_mode="values"):
        if verbose:
            last_message = chunk["messages"][-1]
            
            # Idris spec: LLMResponse (ToolCalls | TextAnswer)
            if hasattr(last_message, 'content') and last_message.content:
                print(f"[{last_message.__class__.__name__}]")
                print(last_message.content)
                print()
            elif hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                # Idris spec: ToolCall in Tools.idr
                for tc in last_message.tool_calls:
                    print(f"🔧 도구 호출: {tc['name']}")
                    print(f"   입력: {tc['args']}")
                print()
    
    # 최종 답변 (Idris spec: finalAnswer in AgentExecutionState)
    final_message = chunk["messages"][-1]
    print(f"\n{'='*70}")
    print("최종 답변:")
    print(f"{'='*70}")
    print(final_message.content)
    print(f"{'='*70}\n")
    
    return chunk

print("✅ 헬퍼 함수 정의 완료")

## 5. 테스트 1: 간단한 주가 조회

In [ ]:
# Idris spec: initialExecState in ReActAgent.idr
result1 = run_agent("삼성전자 현재 주가 알려줘")

## 6. 테스트 2: 복합 분석 (여러 도구 사용)

Idris spec: `ReActCycle` (Thought → Action → Observation)

In [ ]:
result2 = run_agent(
    "삼성전자(005930.KS)의 현재 주가와 20일 이동평균을 비교하고, "
    "최근 관련 뉴스도 검색해서 투자 의견을 제시해줘"
)

## 7. 테스트 3: 미국 주식 분석

In [ ]:
result3 = run_agent(
    "Apple(AAPL) 주식의 기업 정보와 최근 3개월 주가 추이를 분석해줘"
)

## 8. 불변 속성 검증 (Idris spec에서 정의)

Idris에서 정의한 속성들을 Python에서 런타임 검증:

In [ ]:
# ToolAgentState 예시 생성 및 검증
from python.models.tools import ToolHistory, ToolExecution

# Idris spec: initialToolState
state = ToolAgentState.initial_state("테스트 질문", max_calls=5)

# 도구 실행 시뮬레이션
exec1 = ToolExecution(
    tool_name="search_web",
    arguments={"query": "test"},
    result="테스트 결과",
    call_id="call_1"
)
state.tool_history.add_execution(exec1)
state.current_iteration += 1

# Idris spec: ToolCallBound, HistoryConsistent 검증
print("불변 속성 검증:")
print(f"  ToolCallBound: {state.tool_history.total_calls <= state.max_tool_calls}")
print(f"  HistoryConsistent: {len(state.tool_history.executions) == state.tool_history.total_calls}")
print(f"  전체 검증: {state.verify_invariants()}")
print(f"\n현재 상태:")
print(f"  도구 호출 횟수: {state.tool_history.total_calls}/{state.max_tool_calls}")
print(f"  더 호출 가능: {state.can_call_more_tools()}")

## 9. 아키텍처 비교

| 항목 | 기존 (2 web_search.ipynb) | 새로운 (Tool-based) |
|------|---------------------------|---------------------|
| **명세** | 없음 | Idris 형식 명세 |
| **아키텍처** | 커스텀 노드 + 조건부 엣지 | ReAct Prebuilt Agent |
| **도구 정의** | 함수 내부에 하드코딩 | @tool 데코레이터 |
| **확장성** | 새 기능 추가 시 노드/엣지 수정 | 새 도구 추가만으로 확장 |
| **LLM 역할** | 답변 생성만 | 도구 선택 + 답변 생성 |
| **평가** | 수동 구현 (qa_eval) | LLM 자체 판단 |
| **반복 제어** | max_iterations 수동 관리 | Agent 자동 종료 |
| **타입 안전성** | 런타임만 | Idris 컴파일 타임 + Python 런타임 |
| **불변 속성** | 없음 | Idris에서 정의, Python에서 검증 |

### Idris 명세의 이점:

1. **설계 검증**: Python 구현 전에 타입 체크로 로직 검증
2. **문서화**: 타입으로 시스템 동작 명확히 기술
3. **불변 속성**: 런타임 검증할 속성을 명세에 정의
4. **교육**: 학생들이 형식 명세의 중요성 학습

## 10. 연습 문제

### 문제 1: 새로운 도구 추가 (Idris-first 방식)

1. `idris/Domain/Tools.idr`에 RSI 도구 명세 추가
2. `idris2 --check` 실행
3. `python/models/tools.py`에 구현
4. Git commit

```python
@tool
def calculate_rsi(
    ticker: Annotated[str, "주식 티커 심볼"],
    period: Annotated[int, "RSI 계산 기간"] = 14
) -> str:
    """
    RSI 지표를 계산합니다.
    70 이상: 과매수, 30 이하: 과매도
    
    Idris spec: calculateRsiTool in Tools.idr
    """
    # TODO: 구현하세요
    pass
```

### 문제 2: 포트폴리오 분석 도구

여러 주식을 입력받아 포트폴리오 수익률을 계산하는 도구를 Idris 명세부터 작성하세요.

### 문제 3: 불변 속성 추가

`ToolAgentState`에 새로운 불변 속성을 정의하고 검증 함수를 작성하세요.